In [2]:

import numpy as np
import plotly.graph_objects as go
import open3d as o3d
from os import listdir
from os.path import isfile, join
import Plotter
from Skeleton import Skeleton
from Joint import Joint
from Skin import Skin



path_to_mesh = 'G:/My Drive/Research/gaussian_splatting/mesh'
parts = ['body','right_wing','left_wing']
translation = [-0.1-1,0,1]
skin = Skin(parts,path_to_mesh,scale = 1000)
skin.translate_ptcloud_skin(translation)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'fly_model'

In [1]:
# initilize skeletone, joints and bones
pitch_body = 0
joints = {'root_no_bone':[1,0,0],
          'neck':[0.6,0,0.3],
          'neck_thorax': [0.6,0,0.3],
          'head':[0.3,0,0],
          'thorax':[-1,0,0],
          'abdomen':[-1.3,0,0] ,
          'right_sp_no_bone': [-0,0,0.3],
          'right_wing_root':[0,-0.3,0],
          'right_wing_tip':[0,-2.2,0],
          'left_sp_no_bone':[0,0,0.3],
          'left_wing_root':[0,0.3,0],
          'left_wing_tip':[0,2.2,0]
          }

bone_rotation = {'root_no_bone':[0,-pitch_body,0],
          'neck':[0,pitch_body,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[-0,0,0],
          'left_wing_tip':[0,0,0]
          }

parent_child = {'root_no_bone':['neck','neck_thorax','right_sp_no_bone','left_sp_no_bone'],
                      'neck':['head'],
                      'neck_thorax':['thorax'],
                      'thorax':['abdomen'],
                      'right_sp_no_bone':['right_wing_root'],
                      'right_wing_root': ['right_wing_tip'],
                      'left_sp_no_bone': ['left_wing_root'],
                      'left_wing_root':['left_wing_tip']}

skeleton = Skeleton(parent_child,joints,bone_rotation,skin)
# skeleton.initilize_joints(joints)
# skeleton.connect_parent_child(parent_child)
# skeleton.update_local_roattion(local_rotation)
# skeleton.update_global_rotation(rest_bind = True)
# skeleton.get_global_point_skeleton_branch()
# skeleton.add_bone_to_joint()

# skin.calculate_weights(skeleton,constant_weight = [None,'right_wing_root','left_wing_root'],th = 10)




NameError: name 'Skeleton' is not defined

In [3]:
Plotter.plot_skeleton_and_skin_weight(skin.ptcloud_skin,skin,skeleton)

In [4]:
Plotter.plot_skeleton_and_skin_hull(skin.ptcloud_skin,skin,skeleton,opacity = 0.2)

In [5]:
# transform the joints, bones and skin
pitch_body=45
local_rotation = {'root_no_bone':[0,-pitch_body,20],
          'neck':[0,0,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,-0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[30,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[0,0,0],
          'left_wing_tip':[0,0,0]
          }
skeleton.update_local_roattion(local_rotation)
skeleton.update_global_rotation()
skeleton.get_global_point_skeleton_branch()
skeleton.update_bone()


transformed_points, transformed_normals = skin.rotate_skin(skeleton)

In [6]:
Plotter.plot_skeleton_and_skin_hull(transformed_points,skin,skeleton,opacity = 0.1)

In [8]:
Plotter.plot_skeleton_and_skin_normals(transformed_points,skin,skeleton,normals = transformed_normals)
Plotter.plot_skeleton_and_skin_weight(transformed_points,skin,skeleton)